In [1]:
#combine features
import os
import csv

with open("../../3ormoreDATLOW.csv",'r') as f:
        it = csv.reader(f)
        listit = list(it)

presents = {} #map samp to all present mutations
firstrow = [""] #this will list all mutations at the top
#find mutations within various samples, cut off past
for x in range(1,len(listit)):
    inp = listit[x][0]
    presents[inp] = []
    for y in range(1,len(listit[0])-1):
        if(float(listit[x][y]) == 0.0):
            continue
        pieces = listit[0][y].split("_")
        #remake name properly to combined format
        #eg: FBXW7_GRCh37_4:153247289-153247289_MissenseMutationSNP-G-G-C_MissenseMutationSNP-G-A-A
        firstpart = pieces[0] + "_" + pieces[1] + "_" + pieces[2]
        descriptor = ""
        for m in range(3,len(pieces)):
            descriptor += pieces[m] + "-" #TODO: maybe it could be made more readable than this?
        descriptor = descriptor[:-1]
        descriptor = descriptor.replace('_','')
        flag = False
        for k in range(0,len(firstrow)):
            if(firstrow[k] == ""):
                continue
            parts = firstrow[k].split("_")
            newparts = parts[0] + '_' + parts[1] + '_' + parts[2]
            if firstpart == newparts:
                #print(firstpart)
                flag = True #feature is already in set
                if(descriptor in parts):
                    #this mutation is already in the name
                    break
                else:
                    #append new mutation type
                    firstrow[k] = firstrow[k] + "_" + descriptor

        if not flag: #we have a new mutation
            firstrow.append(firstpart+"_"+descriptor)

        if inp in presents:
            presents[inp].append(firstpart)
        else:
            presents[inp] = [firstpart]

        
firstrow

['',
 'BRAF_GRCh37_7:140453136-140453136_Missense-Mutation-SNP-A-A-T_Missense-Mutation-SNP-A-T-T',
 'GRB14_GRCh37_2:165365288-165365296_In-Frame-Del-DEL-TTTTTTTTT----',
 'MBD4_GRCh37_3:129155548-129155557_Frame-Shift-Del-DEL-TTTTTTTTTT----',
 'MSH6_GRCh37_2:48030640-48030647_Frame-Shift-Del-DEL-CCCCCCCC----',
 'PRDM2_GRCh37_1:14108749-14108757_In-Frame-Del-DEL-AAAAAAAAA----',
 'HMMR_GRCh37_5:162917426-162917434_In-Frame-Del-DEL-AAAAAAAAA----',
 'SEC63_GRCh37_6:108214755-108214764_Frame-Shift-Del-DEL-TTTTTTTTTT----',
 'KRAS_GRCh37_12:25398284-25398284_Missense-Mutation-SNP-C-T-T',
 'PIK3CA_GRCh37_3:178952085-178952085_Missense-Mutation-SNP-A-G-G',
 'RAD50_GRCh37_5:131931452-131931460_In-Frame-Del-DEL-AAAAAAAAA----',
 'ACVR2A_GRCh37_2:148683686-148683693_Frame-Shift-Del-DEL-AAAAAAAA----',
 'CASP5_GRCh37_11:104878041-104878050_Frame-Shift-Del-DEL-TTTTTTTTTT----',
 'MSH3_GRCh37_5:79970915-79970922_Frame-Shift-Del-DEL-AAAAAAAA----',
 'ATR_GRCh37_3:142274740-142274749_Frame-Shift-Del-DEL-TTT

In [2]:
firstrow.append("class")
numfeats = len(firstrow) - 2
print("num features",str(numfeats))

#find foreground vs. background
newset = [firstrow]
foreback = {}
for x in range(1,len(listit)):
    if listit[x][0] in foreback:
        print("ERROR.") #multiple of same sample
    else:
        if(float(listit[x][len(listit[0])-1]) == 1.0):
            foreback[listit[x][0]] = "1.0"
        elif(float(listit[x][len(listit[0])-1]) == 2.0):
            foreback[listit[x][0]] = "2.0"
        else:
            foreback[listit[x][0]] = "-1.0"

#attach to new matrix
for key,muts in presents.items():
    nextrow = []
    nextrow.append(key)
    for item in firstrow:
        if(item != "" and item != "class"):
            deconstruct = item.split("_")
            item = (deconstruct[0] + "_" + deconstruct[1] + "_" + deconstruct[2])
            if item in muts:
                nextrow.append("1.0")
            else:
                nextrow.append("0.0")
    nextrow.append(foreback[key])
    newset.append(nextrow)

with open("../../3ormoreDATLOW.csv","w") as fun:
    writer = csv.writer(fun)
    writer.writerows(newset)
    
len(presents)

num features 17


224

In [3]:
#take a look at newly combined features
'''
examine = ["KRAS_GRCh37_12:25398284-25398284_Missense-Mutation-SNP-C-T-T_Missense-Mutation-SNP-C-A-A","BRAF_GRCh37_7:140453136-140453136_Missense-Mutation-SNP-A-A-T_Missense-Mutation-SNP-A-T-T"]
for feature in examine:
    y = newset[0].index(feature)
    fp = tp = 0
    for x in range(1, len(newset)):
        if(float(newset[x][y]) == 1.0):
            if(float(newset[x][len(newset[0])-1]) == -1.0):
                fp += 1
            else:
                tp += 1
    print(feature," FP:",fp," TP:",tp)
'''

'\nexamine = ["KRAS_GRCh37_12:25398284-25398284_Missense-Mutation-SNP-C-T-T_Missense-Mutation-SNP-C-A-A","BRAF_GRCh37_7:140453136-140453136_Missense-Mutation-SNP-A-A-T_Missense-Mutation-SNP-A-T-T"]\nfor feature in examine:\n    y = newset[0].index(feature)\n    fp = tp = 0\n    for x in range(1, len(newset)):\n        if(float(newset[x][y]) == 1.0):\n            if(float(newset[x][len(newset[0])-1]) == -1.0):\n                fp += 1\n            else:\n                tp += 1\n    print(feature," FP:",fp," TP:",tp)\n'